In [ ]:
import numpy as np
import json
from keras.models import model_from_json
from sklearn.model_selection import train_test_split



def save_model(model, name):
    model_json = model.to_json()
    with open("{}.json".format(name), "w") as json_file:
        json_file.write(model_json)
    model.save_weights("{}.h5".format(name))
    print("Model {} saved!".format(name))
    
def load_model(name):
    json_file = open('{}.json'.format(name), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("{}.h5".format(name))
    return loaded_model

# static_model = load_model("static_model")


dynamic_gestures_labesl_dict = {0: "stop",
                                1: "come_on",
                                2: "go_away",
                                3: "look_at_me",
                                4: "other"}

gestures_to_num_dict = dict(zip(dynamic_gestures_labesl_dict.values(), dynamic_gestures_labesl_dict.keys()))

full_data = []
full_labels = []

for gesture in gestures_to_num_dict:
    fname = "{}.npy".format(gesture)
    data = np.load(fname, allow_pickle=True)
    label = gestures_to_num_dict[gesture]
    for single_data in data:
        if len(single_data) == 63:
            full_data.append(np.array(single_data[5:]))
            full_labels.append(label)
    
full_data = np.array(full_data)
full_labels = np.array(full_labels)


labels_dict = {"hands_down": 0,
               "stop": 1,
               "hands_up": 2,
               "hands_up_small": 3,
               "hands_down_small": 4,
               "hads_down_up": 5,
               "hands_to_sides": 6}

decode_labels_dict = {val: key for key, val in labels_dict.items()}

def get_static_gestures_list(features_sequence):
    static_gestures_numbers = []
    for features in features_sequence:
        data = np.array([features])
        prediction_result = np.argmax(static_model.predict(data))
        static_gestures_numbers.append(prediction_result)
    return static_gestures_numbers


static_gestures_sequence_lists = []

for data in full_data:
    static_gestures_sequence = get_static_gestures_list(data)
    data = np.array([static_gestures_sequence])
    data = np.hstack(data)
    static_gestures_sequence_lists.append(data)

    



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(static_gestures_sequence_lists, full_labels, stratify=full_labels,
                                                    test_size=0.3, random_state=42)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape(704, 58, 1)
X_test = X_test.reshape(302, 58, 1)

In [ ]:
X_test.shape


In [ ]:
X_train.shape

##### X_train[0].shape[-1]

# 1D Convolution

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop


input_shape=(None, X_train[0].shape[-1])

conv_1d_model = Sequential()
conv_1d_model.add(layers.Conv1D(58, 5, activation='sigmoid', input_shape=input_shape))
# conv_1d_model.add(layers.MaxPooling1D(3))
conv_1d_model.add(layers.Conv1D(32, 5, activation='sigmoid'))
conv_1d_model.add(layers.MaxPooling1D(3))
conv_1d_model.add(layers.Conv1D(32, 5, activation='sigmoid'))#6.4. Обработка последовательностей с помощью сверточных нейронных сетей   265
conv_1d_model.add(layers.GlobalMaxPooling1D())
conv_1d_model.add(layers.Dense(5, activation="softmax"))
conv_1d_model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["acc"])

history_1d = conv_1d_model.fit(X_train,y_train, epochs=500, validation_data=(np.array(X_test), np.array(y_test)), verbose=False)

In [ ]:
acc = history_1d.history["acc"]
val_acc = history_1d.history["val_acc"]

In [ ]:
np.argmax(val_acc)

In [ ]:
np.max(val_acc)

# RNN

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

rnn_model = Sequential()

# rnn_model.add(Embedding(max_features, 32))
rnn_model.add(SimpleRNN(58, input_shape=input_shape))
rnn_model.add(Dense(5, activation='softmax'))

rnn_model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["acc"])

history_rnn = rnn_model.fit(X_train,y_train, epochs=500, validation_data=(np.array(X_test), np.array(y_test)), verbose=False)

In [ ]:
rnn_acc = history_rnn.history["acc"]
rnn_val_acc = history_rnn.history["val_acc"]

In [ ]:
np.argmax(rnn_val_acc)

In [ ]:
np.max(rnn_val_acc)

# LSTM

In [ ]:
from keras.layers import LSTM, Dense

from keras.models import Sequential

lstm_model = Sequential()

# lstm_model.add(Embedding(max_features, 32))
lstm_model.add(LSTM(58, input_shape=input_shape))
lstm_model.add(Dense(5, activation='softmax'))

lstm_model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["acc"])

history_lstm = lstm_model.fit(X_train,y_train, epochs=500, validation_data=(np.array(X_test), np.array(y_test)), verbose=False)

In [ ]:
lstm_acc = history_lstm.history["acc"]
lstm_val_acc = history_lstm.history["val_acc"]

In [ ]:
np.argmax(lstm_val_acc)

In [ ]:
np.max(lstm_val_acc)

In [ ]:
X_test[0].shape

# GRU

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

gru_model = Sequential()
gru_model.add(layers.GRU(58,input_shape=input_shape))

gru_model.add(Dense(5, activation='softmax'))

gru_model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["acc"])

history_gru = gru_model.fit(X_train,y_train, epochs=500, validation_data=(np.array(X_test), np.array(y_test)), verbose=False)

In [ ]:
gru_acc = history_gru.history["acc"]
gru_val_acc = history_gru.history["val_acc"]
np.argmax(gru_val_acc)
np.max(gru_val_acc)

In [ ]:
np.argmax(gru_val_acc)

In [ ]:
np.max(gru_val_acc)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

gru2_model = Sequential()
gru2_model.add(layers.GRU(58,
                            dropout=0.1,
                            recurrent_dropout=0.5,
                            return_sequences=False,
                            input_shape=input_shape))
# gru2_model.add(layers.GRU(58), activation="sigmoid")
gru2_model.add(Dense(5, activation='softmax'))

gru2_model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["acc"])

history_gru2 = gru2_model.fit(X_train,y_train, epochs=500, validation_data=(np.array(X_test), np.array(y_test)), verbose=False)

In [ ]:
gru2_acc = history_gru2.history["acc"]
gru2_val_acc = history_gru2.history["val_acc"]
np.argmax(gru2_val_acc)
np.max(gru2_val_acc)

In [282]:
import cv2
from src.PoseExtractor import PoseExtractor
from src.data_processing.pose_utils import *
from keras.models import model_from_json
import json


labels_dict = {"hands_down": 0,
               "stop": 1,
               "hands_up": 2,
               "hands_up_small": 3,
               "hands_down_small": 4,
               "hads_down_up": 5,
               "hands_to_sides": 6}

decode_labels_dict = {val: key for key, val in labels_dict.items()}


dynamic_gestures_labesl_dict = {0: "stop",
                                1: "come_on",
                                2: "go_away",
                                3: "look_at_me",
                                4: "other"}

gestures_to_num_dict = dict(zip(dynamic_gestures_labesl_dict.values(), dynamic_gestures_labesl_dict.keys()))


def save_model(model, name):
    model_json = model.to_json()
    with open("{}.json".format(name), "w") as json_file:
        json_file.write(model_json)
    model.save_weights("{}.h5".format(name))
    print("Model {} saved!".format(name))
    
def load_model(name):
    json_file = open('{}.json'.format(name), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("{}.h5".format(name))
    return loaded_model

static_model = load_model("static_model")
dynamic_model = load_model("dynamic_model")



def get_biggest_pose(poses):
    max_distance = -1
    biggest_pose = None
    if len(poses):
        for pose in poses:
            pose_key_point1 = pose.points[1]
            pose_key_point2 = pose.points[8]
            if all(pose_key_point1.int_xy) and all(pose_key_point2.int_xy):
                distance = get_distance_between_points(pose_key_point1, pose_key_point2)
                if distance >= max_distance:
                    max_distance = distance
                    biggest_pose = pose
        return biggest_pose
    
    
def find_distance(pt1, pt2):
    return np.sqrt((pt2.x - pt1.x)**2 + (pt2.y - pt1.y)**2)

    
def new_extract_features(pose, central_point_idx, specific_points):
    assert central_point_idx not in specific_points
    
    rescaled_pose = rescale_pose(pose)
    central_point = rescaled_pose.points[central_point_idx]
    
    features = []
    
    for current_points_idx in specific_points:
        current_point = rescaled_pose.points[current_points_idx]

        dx = central_point.x - current_point.x
        dy = central_point.y - current_point.y
    
        features.append(dx)
        features.append(dy)
    
    return np.array(features)


class FeaturesSequence():
    
    default_pose = 0
    
    def __init__(self, max_features_num):
        self.__max_features_num = max_features_num
        self.__features_sequence = [0] * self.__max_features_num
    
    def update_features_sequense(self, new_features):
        if len(self.__features_sequence) >= self.__max_features_num:
            self.__features_sequence = self.__features_sequence[1:]
        self.__features_sequence.append(new_features)
        
    def get_vectorized_features(self):
        return self.__features_to_vec(self.__features_sequence).reshape(1, -1)
        
        
    @property
    def features(self):
        return self.__features_sequence
    
    @staticmethod
    def __features_to_vec(features):
        vectors = []
        for f in features:
            zeros = np.zeros((6))
            zeros[f-1] = 1
            vectors.append(zeros)
        return np.array(vectors)
    
    
capture = cv2.VideoCapture(0)
pose_extractor = PoseExtractor()
top_pose_points = [0, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 16, 17, 18]

features_seq = FeaturesSequence(58)

while True:
    ret, img = capture.read()
    if not ret:
        break
        
    poses = pose_extractor.extract_poses_from_image(img)
    actual_pose =  get_biggest_pose(poses)
    if actual_pose is not None:
        features = new_extract_features(actual_pose, 1, top_pose_points)
        gesture = static_model.predict(features.reshape(1, -1))
        gesture = np.argmax(gesture)
        str_gesture = decode_labels_dict[gesture]
        
        features_seq.update_features_sequense(gesture)
#         features_seq.update_features_sequense(gesture)
#         features_seq.update_features_sequense(gesture)
#         features_seq.update_features_sequense(gesture)
#         features_seq.update_features_sequense(gesture)
#         features_seq.update_features_sequense(gesture)
#         features_seq.update_features_sequense(gesture)
        features_seq.update_features_sequense(gesture)


#         vec = features_seq.get_vectorized_features()
        
#         print(vec)
        features = np.array([np.array(features_seq.features).reshape(-1, 1)])
#         print(features.shape)
        dynamic_gesture = lstm_model.predict([features])
        dynamic_gesture = np.argmax(dynamic_gesture)
        str_dynamic_gesture = dynamic_gestures_labesl_dict[dynamic_gesture]
        
        cv2.putText(img, str_dynamic_gesture, (20, 20), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 3)
        
        
        for point in actual_pose.points:
            center = (int(point.x), int(point.y))
            cv2.circle(img, center, 3, (0, 255, 255), -1)
            
    cv2.namedWindow("image", cv2.WINDOW_NORMAL)
    cv2.imshow("image", img)
    k = cv2.waitKey(10)
    if k & 0xFF == 27:
        break
    
    
capture.release()
cv2.destroyAllWindows()

In [ ]:
X_train.shape

In [ ]:
capture.release()
cv2.destroyAllWindows()

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence
max_features = 10000
max_len = 500
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=
max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
# x_train = sequence.pad_sequences(x_train, maxlen=max_len)
# x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [ ]:
y_train

In [ ]:
full_data.shape

In [ ]:
vec_full_data = full_data.reshape(1006, 58*28)

In [ ]:
from sklearn.model_selection import train_test_split
full_X_train, full_X_test, full_y_train, full_y_test = train_test_split(vec_full_data, full_labels, stratify=full_labels,
                                                                        test_size=0.3, random_state=42)

In [ ]:
full_X_train.shape

In [ ]:
full_X_test.shape

In [ ]:
full_X_train = full_X_train.reshape(704, 1624, 1)
full_X_test = full_X_test.reshape(302, 1624, 1)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

# l = layers.Conv1D
input_shape=(None, full_X_train[0].shape[-1])

full_conv_1d_model = Sequential()
full_conv_1d_model.add(layers.Conv1D(100, 5, activation='sigmoid', input_shape=input_shape))
full_conv_1d_model.add(layers.Conv1D(100, 5, activation='sigmoid'))
full_conv_1d_model.add(layers.MaxPooling1D(3))
full_conv_1d_model.add(layers.Conv1D(100, 5, activation='sigmoid'))#6.4. Обработка последовательностей с помощью сверточных нейронных сетей   265
full_conv_1d_model.add(layers.GlobalMaxPooling1D())
full_conv_1d_model.add(layers.Dense(5, activation="softmax"))
full_conv_1d_model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["acc"])

full_history_1d = full_conv_1d_model.fit(full_X_train, full_y_train, epochs=100,
                                         validation_data=(np.array(full_X_test), np.array(full_y_test)),
                                         verbose=True)

In [262]:
max(full_history_1d.history["val_acc"])

0.9867549538612366

In [263]:
np.argmax(full_history_1d.history["val_acc"])

70

In [275]:
capture.release()
cv2.destroyAllWindows()